# Metodo de Newton para solucionar el problema de Markowitz

**Responsable:**
Itzel Muñoz

**Infraestructura usada:** instancia en AWS, con imagen de Docker

**Fuente:** código desarrollada en etapas previas

## Objetivo:

Utilizar el método de Newton visto en clase, utilizando tanto diferenciación aproximada y funciones simbólicas, para dar solución al problema de minimización de riesgo en una cartera de inversión. 

Recordemos que el problema se puede plantear como: 

$$\min_{w}  \frac{1}{2} w^t \Sigma w$$

sujeto a:

$$ w^t \mu= r$$

$$ w^t 1_{m}= 1$$

En donde $\Sigma$ es la matriz de covarianzas asociadas a los rendimientos de los activos en el periodo de interés.

## Importación de librerías

Las librerías son en su mayoría las construidas para el proyecto, así como numpy y cupy.

In [27]:
import numpy as np
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt
#from utils import inc_index, dec_index, compute_error, norm_residual, condicion_cupy
#from line_search import line_search_by_backtracking, line_search_for_residual_by_backtracking

## Cargamos la información y calculamos los parámetros

Primero cargamos los precios de las acciones que se han elegido para esta implementación

In [3]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [4]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 completed


Usando las funciones auxiliares definidas previamente calculamos los rendimientos esperados y la matriz de varianza y covarianza para las acciones

In [5]:
mu = aux.calcular_rendimiento(datos)

In [6]:
S = aux.calcular_varianza(datos)

Calcularemos cuál fue el rendimiento máximo obtenido (de entre las medias de los datos históricos) y lo usaremos como el rendimiento deseado en el problema de optimización

In [7]:
r=max(mu).item()
r

0.40221087876214845

## Resolvemos con el Método de Newton usando diferencias finitas

La función a optimizar será la siguiente:

In [8]:
fo = lambda w: w@S@w

Usamos la solución analítica desarrollada en la fase 1 del proyecto, para tener la solución con la cual comparar.

In [9]:
w_ast = mkv.markowitz(r,mu,S)

La matriz $A$ de restricciones está dada por:

In [10]:
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
A

array([[ 0.13254293,  0.02782955,  0.25778345, -0.01787111, -0.02008539,
        -0.02286483,  0.19078421,  0.05743478,  0.17062275,  0.40221088,
         0.13618597,  0.02364206,  0.0476638 ,  0.13217291, -0.09666644,
        -0.03426582,  0.15880965,  0.17870289,  0.18557566,  0.15042008,
         0.08379133,  0.07154137,  0.08843336,  0.02202303,  0.07353272,
         0.13731909,  0.23605478,  0.19942876,  0.06806558,  0.26292508,
         0.07357149,  0.06803816,  0.03867589,  0.05090149,  0.14401416,
        -0.09339292,  0.23072551,  0.05036842,  0.05587177, -0.01264577,
        -0.01474117,  0.20678446,  0.06274015, -0.02729424,  0.1990038 ,
         0.07054765,  0.06563628,  0.04203765,  0.07174071, -0.13227261],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.

Por otro lado el vector $b$ está dado por:

In [11]:
b = cp.array([r,1])
b

array([0.40221088, 1.        ])

Definimos el punto inicial con la función definida anteriormente:

In [12]:
w_0 = utils.feasible_markowitz(r,mu)

Definimos los parámetros para el método de Newton de la siguiente manera:

In [14]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

Ejecutamos el método de newton:

In [15]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)



I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0014	0.0007	1.0802	3.7098	---		9792.2972
1	0.0014	0.0	0.062	0.0002	1	9792.2972
2	0.0014	0.0	0.0014	0.0	1	9792.2972
Error of x with respect to x_ast: 0.0013645454843084779
Approximate solution: [ 1.58451472e-01 -2.08715680e-02  1.58049916e-01 -9.28741024e-02
  3.17119956e-02  5.10796562e-02  8.71482821e-02  1.26558868e-02
  3.72546328e-02  2.86008401e-01 -5.99170671e-03  2.23685830e-03
  2.03058504e-01  9.46156284e-02  2.28692346e-02  1.49747220e-02
  7.59212363e-03  2.96653985e-02  5.76128693e-02  1.98112969e-01
  1.19939351e-01  1.27836405e-01  1.41264175e-01  1.36407079e-02
  8.83738046e-02  1.50456411e-01  1.69247166e-01  7.71830489e-02
  8.08890475e-02  8.24456605e-02  1.92202159e-01 -2.40766179e-02
  2.64525535e-02  7.69707517e-02  2.20854988e-02 -1.08195183e-01
  1.64041534e-01  1.77232693e-02  6.16431612e-02 -1.08211841e-01
 -5.03241268e-02  1.38816492e-01  1.03596602e-01 -4.28402611e-02
  1.07684600e-02 -

La solución dada por el Método de Newton usando las diferenciación numérica es muy similar a la del método cerrado. Siendo el error de:

In [16]:
Err_plot[-1]

7.794891865124757e-08

In [17]:
w

array([ 1.58451472e-01, -2.08715680e-02,  1.58049916e-01, -9.28741024e-02,
        3.17119956e-02,  5.10796562e-02,  8.71482821e-02,  1.26558868e-02,
        3.72546328e-02,  2.86008401e-01, -5.99170671e-03,  2.23685830e-03,
        2.03058504e-01,  9.46156284e-02,  2.28692346e-02,  1.49747220e-02,
        7.59212363e-03,  2.96653985e-02,  5.76128693e-02,  1.98112969e-01,
        1.19939351e-01,  1.27836405e-01,  1.41264175e-01,  1.36407079e-02,
        8.83738046e-02,  1.50456411e-01,  1.69247166e-01,  7.71830489e-02,
        8.08890475e-02,  8.24456605e-02,  1.92202159e-01, -2.40766179e-02,
        2.64525535e-02,  7.69707517e-02,  2.20854988e-02, -1.08195183e-01,
        1.64041534e-01,  1.77232693e-02,  6.16431612e-02, -1.08211841e-01,
       -5.03241268e-02,  1.38816492e-01,  1.03596602e-01, -4.28402611e-02,
        1.07684600e-02, -2.33063166e+00,  1.87582962e-01,  2.63266188e-01,
        1.20059552e-01, -1.05566446e-01])

In [18]:
w_ast

array([ 1.58450467e-01, -2.08710167e-02,  1.58051608e-01, -9.28971234e-02,
        3.17093854e-02,  5.10824568e-02,  8.71800652e-02,  1.26389401e-02,
        3.72861663e-02,  2.86000502e-01, -5.98809816e-03,  2.24204759e-03,
        2.03075862e-01,  9.46031116e-02,  2.28767051e-02,  1.49919971e-02,
        7.60705871e-03,  2.96676614e-02,  5.76521210e-02,  1.98109871e-01,
        1.19928211e-01,  1.27869506e-01,  1.41300443e-01,  1.36285373e-02,
        8.83904595e-02,  1.50479883e-01,  1.69293473e-01,  7.72036844e-02,
        8.09120869e-02,  8.24658797e-02,  1.92197926e-01, -2.40096192e-02,
        2.64375674e-02,  7.69647345e-02,  2.20741569e-02, -1.08207563e-01,
        1.64032735e-01,  1.77020958e-02,  6.16398189e-02, -1.08210792e-01,
       -5.03462779e-02,  1.38834333e-01,  1.03567407e-01, -4.28198239e-02,
        1.06872828e-02, -2.33314837e+00,  1.89811818e-01,  2.63296460e-01,
        1.20089776e-01, -1.05535620e-01])

La varianza entre ambos métodos también es muy similar

In [19]:
w@S@w

array(9.45979293e-05)

In [20]:
w_ast@S@w_ast

array(9.45979219e-05)

In [21]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(7.79489126e-08)

Además cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [22]:
w@mu

array(0.40221088)

In [23]:
utils.compute_error(w@mu, r)

array(1.38015042e-16)

In [25]:
sum(w)

array(1.)

## Resolvemos con el Método de Newton usando funciones simbólicas

Ahora utilizaremos de nuevo el método de Newton usando las funciones simbólicas:

In [26]:
import inspect

In [28]:
lines = inspect.getsource(opt.gfo_cp_mark)
print(lines)

def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block



In [29]:
lines = inspect.getsource(opt.Hfo_cp_mark)
print(lines)

def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block



Ejecutamos el método de newton con las funciones simbólicas:

In [30]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0007	0.0004	1.0802	3.7098	---		10080.8013
1	0.0007	-0.0	0.0	0.0	1	10080.8013
Error of x with respect to x_ast: 2.889517336037216e-12
Approximate solution: [ 1.58450467e-01 -2.08710167e-02  1.58051608e-01 -9.28971234e-02
  3.17093854e-02  5.10824568e-02  8.71800652e-02  1.26389401e-02
  3.72861663e-02  2.86000502e-01 -5.98809816e-03  2.24204759e-03
  2.03075862e-01  9.46031116e-02  2.28767051e-02  1.49919971e-02
  7.60705871e-03  2.96676614e-02  5.76521210e-02  1.98109871e-01
  1.19928211e-01  1.27869506e-01  1.41300443e-01  1.36285373e-02
  8.83904595e-02  1.50479883e-01  1.69293473e-01  7.72036844e-02
  8.09120869e-02  8.24658797e-02  1.92197926e-01 -2.40096192e-02
  2.64375674e-02  7.69647345e-02  2.20741569e-02 -1.08207563e-01
  1.64032735e-01  1.77020958e-02  6.16398189e-02 -1.08210792e-01
 -5.03462779e-02  1.38834333e-01  1.03567407e-01 -4.28198239e-02
  1.06872828e-02 -2.33314837e+00  1.89811818e-01  2.6329

La solución dada por el Método de Newton usando las funciones simbólicas es prácticamente igual al método cerrado, siendo el error de:

In [31]:
Err_plot[-1]

2.1489680023952056e-15

In [32]:
w

array([ 1.58450467e-01, -2.08710167e-02,  1.58051608e-01, -9.28971234e-02,
        3.17093854e-02,  5.10824568e-02,  8.71800652e-02,  1.26389401e-02,
        3.72861663e-02,  2.86000502e-01, -5.98809816e-03,  2.24204759e-03,
        2.03075862e-01,  9.46031116e-02,  2.28767051e-02,  1.49919971e-02,
        7.60705871e-03,  2.96676614e-02,  5.76521210e-02,  1.98109871e-01,
        1.19928211e-01,  1.27869506e-01,  1.41300443e-01,  1.36285373e-02,
        8.83904595e-02,  1.50479883e-01,  1.69293473e-01,  7.72036844e-02,
        8.09120869e-02,  8.24658797e-02,  1.92197926e-01, -2.40096192e-02,
        2.64375674e-02,  7.69647345e-02,  2.20741569e-02, -1.08207563e-01,
        1.64032735e-01,  1.77020958e-02,  6.16398189e-02, -1.08210792e-01,
       -5.03462779e-02,  1.38834333e-01,  1.03567407e-01, -4.28198239e-02,
        1.06872828e-02, -2.33314837e+00,  1.89811818e-01,  2.63296460e-01,
        1.20089776e-01, -1.05535620e-01])

In [33]:
w_ast

array([ 1.58450467e-01, -2.08710167e-02,  1.58051608e-01, -9.28971234e-02,
        3.17093854e-02,  5.10824568e-02,  8.71800652e-02,  1.26389401e-02,
        3.72861663e-02,  2.86000502e-01, -5.98809816e-03,  2.24204759e-03,
        2.03075862e-01,  9.46031116e-02,  2.28767051e-02,  1.49919971e-02,
        7.60705871e-03,  2.96676614e-02,  5.76521210e-02,  1.98109871e-01,
        1.19928211e-01,  1.27869506e-01,  1.41300443e-01,  1.36285373e-02,
        8.83904595e-02,  1.50479883e-01,  1.69293473e-01,  7.72036844e-02,
        8.09120869e-02,  8.24658797e-02,  1.92197926e-01, -2.40096192e-02,
        2.64375674e-02,  7.69647345e-02,  2.20741569e-02, -1.08207563e-01,
        1.64032735e-01,  1.77020958e-02,  6.16398189e-02, -1.08210792e-01,
       -5.03462779e-02,  1.38834333e-01,  1.03567407e-01, -4.28198239e-02,
        1.06872828e-02, -2.33314837e+00,  1.89811818e-01,  2.63296460e-01,
        1.20089776e-01, -1.05535620e-01])

La varianza entre ambos métodos también es prácticamente la misma

In [34]:
w@S@w

array(9.45979219e-05)

In [35]:
w_ast@S@w_ast

array(9.45979219e-05)

In [36]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(2.148968e-15)

Y de igual manera cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [37]:
w@mu

array(0.40221088)

In [38]:
utils.compute_error(w@mu, r)

array(5.52060167e-16)

In [39]:
sum(w)

array(1.)

## Conclusiones

Aunque con ambas variabes se obtienen resultados bastante acertados, como es de esperarse al usar función simbólica, el resultado tiene mayor precisión.